In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

train_data = pd.read_parquet("/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/dataset_v2/train.parq", engine="pyarrow")

#New features
#https://colab.research.google.com/drive/1M9Hh3YWE5jiA59oerCf6fChVYaFJq3Gp?usp=sharing
one_hot_attack_types_df = pd.read_csv("/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/Features/one_hot_attack_types_df.csv", engine="pyarrow")  #TP2-Features-One-hot attack_type.ipynb
one_hot_attack_service_df = pd.read_csv("/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/Features/one_hot_attack_service_df.csv", engine="pyarrow") #TP2-Features-One-hot attack_type.ipynb

#https://colab.research.google.com/drive/16Chf3GEfKOTpDEWehhCE74wFfAcuxWW3?usp=sharing
one_hot_days_df = pd.read_csv("/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/Features/one_hot_days_df.csv", engine="pyarrow") #TP2-Features-One-hot attack_time(days).ipynb

#https://colab.research.google.com/drive/1FAxyoJHtQCsi5PqKHCGXfklz_AENcaTm?usp=sharing
one_hot_hours_df = pd.read_csv("/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/DFs Base Line/one_hot_hours_df.csv", engine="pyarrow") #TP2-Features-One-hot attack_time(hour).ipynb

#https://colab.research.google.com/drive/11Kn4qbTRQ2PmKyYBaApSdACwB2hoCogO?usp=sharing
ports_features_df = pd.read_csv("/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/Features/ports_features_df.csv", engine="pyarrow") #TP2-Features-One-hot open_ports.ipynb

#https://colab.research.google.com/drive/1EKOTO-a7iGCuDPDbBa-casQY9oJCXeg0?usp=sharing
protocols_features_df = pd.read_csv("/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/New Features/protocols_features_df.csv", engine="pyarrow") #TP2-Features-protocol_count.ipynb


Mounted at /content/drive


In [ ]:
train_data.head()

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label
0,2023-07-31 07:17:51+00:00,DE,34011.0,Host Europe GmbH,TR,47721.0,Murat Aktas,http:exploit,0,6466,0
1,2023-07-31 07:17:51+00:00,DE,34011.0,Host Europe GmbH,TR,47721.0,Murat Aktas,http:spam,0,6466,0
2,2023-07-31 07:17:49+00:00,DE,20886.0,bn:t Blatzheim Networks Telecom GmbH,DE,51167.0,Contabo GmbH,http:bruteforce,2,4637,0
3,2023-07-31 07:17:49+00:00,DE,20886.0,bn:t Blatzheim Networks Telecom GmbH,DE,51167.0,Contabo GmbH,http:spam,2,4637,0
4,2023-07-31 07:17:49+00:00,DE,20886.0,bn:t Blatzheim Networks Telecom GmbH,DE,51167.0,Contabo GmbH,http:exploit,2,4637,0


## Preparación del dataset

In [ ]:
label_df = train_data.drop_duplicates(["attacker_ip_enum", "label"]).set_index("attacker_ip_enum").loc[:, ['watcher_as_num','attacker_as_num','attacker_country','watcher_country',"label"]]

label_df['watcher_as_num'] = label_df['watcher_as_num'].astype('category')
label_df['attacker_as_num'] = label_df['attacker_as_num'].astype('category')

dataset = pd.concat(
    [
        one_hot_attack_service_df,
        one_hot_attack_types_df,
        one_hot_days_df,
        ports_features_df,
        protocols_features_df,
        one_hot_hours_df,
        label_df
    ],
    axis=1,
    join="inner",
)

if 'attacker_ip_enum' in dataset.columns:
    dataset = dataset.drop('attacker_ip_enum', axis=1)


In [ ]:
dataset.head()

,database,ftp,http,pop3/imap,sip,smb,ssh,tcp,telnet,unknown_service,...,7,8,9,10,11,watcher_as_num,attacker_as_num,attacker_country,watcher_country,label
0,0,0,1,0,0,0,0,0,0,0,...,10,18,6,16,2,7162.0,14618.0,US,BR,0
1,0,0,1,0,0,0,0,0,0,0,...,26,0,0,0,0,203417.0,27699.0,BR,PL,0
2,0,0,1,0,0,0,0,0,0,0,...,0,1,0,7,3,208857.0,3320.0,DE,AT,0
3,0,0,1,0,0,0,0,0,0,0,...,3,0,3,6,0,20473.0,21826.0,VE,SG,0
4,0,0,1,0,0,0,0,0,0,0,...,0,10,6,18,28,24940.0,25019.0,SA,DE,0


In [ ]:
dataset['label'].value_counts()

0    107293
1      2146
Name: label, dtype: int64

In [ ]:
dataset.columns

Index(['database', 'ftp', 'http', 'pop3/imap', 'sip', 'smb', 'ssh', 'tcp',
       'telnet', 'unknown_service', 'windows', 'bruteforce', 'crawl',
       'exploit', 'scan', 'spam', 'unknown_type', 'Friday', 'Monday',
       'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'port_1080',
       'port_110', 'port_1194', 'port_143', 'port_1701', 'port_1723',
       'port_20', 'port_21', 'port_22', 'port_23', 'port_25', 'port_500',
       'port_53', 'port_8080', 'open_ports_count', 'protocol_tcp',
       'protocol_udp', 'protocols_count', '0', '1', '2', '3', '4', '5', '6',
       '7', '8', '9', '10', '11', 'watcher_as_num', 'attacker_as_num',
       'attacker_country', 'watcher_country', 'label'],
      dtype='object')

## Train y test

In [ ]:
!pip install --upgrade imbalanced-learn scikit-learn
from sklearn.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 20.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.10.1
    Uninstalling imbalanced-learn-0.10.1:
      Successfully uninstalled imbalanced-learn-0.10.1


Luego del concat, el dataset resultante tiene como index a attacker_ip_enum, sin repeticiones

In [ ]:
train, test = train_test_split(dataset, test_size=0.3, random_state=42)

## Mean encoding de 'attacker_as_num' y 'watcher_as_num'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


# Calcular las medias por categoría en el conjunto de entrenamiento
means_attacker_as_num = train.groupby('attacker_as_num')['label'].mean()
means_watcher_as_num = train.groupby('watcher_as_num')['label'].mean()

train['attacker_as_num'] = train['attacker_as_num'].map(means_attacker_as_num)
test['attacker_as_num'] = test['attacker_as_num'].map(means_attacker_as_num)

train['watcher_as_num'] = train['watcher_as_num'].map(means_watcher_as_num)
test['watcher_as_num'] = test['watcher_as_num'].map(means_watcher_as_num)

In [ ]:
train.head(10)

,database,ftp,http,pop3/imap,sip,smb,ssh,tcp,telnet,unknown_service,...,7,8,9,10,11,watcher_as_num,attacker_as_num,attacker_country,watcher_country,label
45062,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0.000000,0.000638,US,AR,0
136282,0,0,1,0,0,0,0,0,0,0,...,2,2,0,0,0,0.028419,0.000000,JP,JP,0
4285,0,0,1,0,0,0,0,0,0,0,...,6,13,5,15,26,0.029615,0.002558,CN,SG,0
3477,0,0,1,0,0,0,0,0,0,0,...,381,361,320,348,360,0.004882,0.000000,AT,AT,0
84069,0,0,1,0,0,0,0,0,0,0,...,6,0,0,0,0,0.000000,0.000000,US,FR,0
23152,0,0,1,0,0,0,0,0,0,0,...,24,0,2,9,0,0.000000,0.000000,IN,PL,0
112485,0,0,1,0,0,0,0,0,0,0,...,3,17,8,19,6,0.024994,0.000000,BR,US,0
11274,0,0,1,0,0,0,0,0,0,0,...,24,33,44,21,79,0.001248,0.000000,DE,DE,0
60830,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0.023983,0.000000,DE,US,0
67344,0,0,1,0,0,0,0,0,0,0,...,4,6,2,0,0,0.018787,0.000000,US,US,0


## Separacion en x e y

In [ ]:
X_train_raw = train.drop(["label"], axis=1)
y_train_raw = train["label"]
X_test_raw = test.drop(["label"], axis=1)
y_test_raw = test["label"]

## Binary encoding de 'attacker_country' y 'watcher_country' e imputacion de valores nulos

In [ ]:
pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.8 MB/s eta 0:00:00


In [ ]:
from category_encoders.binary import BinaryEncoder
from sklearn.impute import SimpleImputer

# Como los valores nan son categóricos, los voy a reemplazar con los más comunes
columns_with_nans = ['attacker_country', 'watcher_country', 'watcher_as_num', 'attacker_as_num']

# Imputar los valores nulos
imputer = SimpleImputer(strategy='most_frequent')
X_train_raw[columns_with_nans] = imputer.fit_transform(X_train_raw[columns_with_nans])
X_test_raw[columns_with_nans] = imputer.transform(X_test_raw[columns_with_nans])

# Crear una nueva columna combinando "attacker_country" y "watcher_country"
X_train_raw['combined_countries'] = X_train_raw['attacker_country'] + '-' + X_train_raw['watcher_country']
X_test_raw['combined_countries'] = X_test_raw['attacker_country'] + '-' + X_test_raw['watcher_country']

# Crear un objeto BinaryEncoder
encoder = BinaryEncoder(cols=['combined_countries','watcher_country','attacker_country'], drop_invariant=True)

# Aplicar el BinaryEncoder a la nueva columna
X_train_enc = encoder.fit_transform(X_train_raw)
X_test_enc = encoder.transform(X_test_raw)



In [ ]:

X_train = X_train_enc
y_train = y_train_raw

X_test = X_test_enc
y_test = y_test_raw

## Modelo

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

est1 = RandomForestClassifier()



est1.fit(X_train, y_train)

y_pred = est1.predict(X_test)

f1_score(y_pred=y_pred, y_true=y_test)

0.39289678135405104

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint


param_dist = {
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['auto', 'sqrt', 'log2', None],
    'max_depth': [None] + list(range(40, 55)),
    'class_weight': [ 'balanced_subsample'],
    'bootstrap': [True, False],
    'n_estimators': randint(190, 220),
    'random_state': [123]
}


random_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(),
    param_distributions=param_dist,
    n_iter=5,
    scoring='f1',
    cv=5,
    random_state=123
)


random_search.fit(X_train, y_train)


best_params = random_search.best_params_
best_estimator = random_search.best_estimator_

print("Mejores Hiperparámetros:", best_params)

y_pred = best_estimator.predict(X_test)

f1 = f1_score(y_true=y_test_raw, y_pred=y_pred)

print("Puntaje F1 en el conjunto de prueba:", f1)


Mejores Hiperparámetros: {'bootstrap': True, 'class_weight': 'balanced_subsample', 'max_depth': 40, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 210, 'random_state': 123}
Puntaje F1 en el conjunto de prueba: 0.6019273535952557


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


mejores_hiperparametros = {'bootstrap': True, 'class_weight': 'balanced_subsample', 'max_depth': 40, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 210, 'random_state': 123}


est2 = RandomForestClassifier(**mejores_hiperparametros)


y_train = y_train_raw
est2.fit(X_train, y_train)

y_pred = est2.predict(X_test)

f1_score(y_pred=y_pred, y_true=y_test)

0.6019273535952557

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


mejores_hiperparametros = {
    'min_samples_split': 5,#4
    'min_samples_leaf': 4,
    'max_features': None,
    'max_depth': 40,#35
    'class_weight': 'balanced_subsample',
    'bootstrap': True,
    'n_estimators': 200,#210
     'random_state': 123
}


est3 = RandomForestClassifier(**mejores_hiperparametros)


y_train = y_train_raw
est3.fit(X_train, y_train)

y_pred = est3.predict(X_test)

f1_score(y_pred=y_pred, y_true=y_test_raw)

0.6044444444444445

## Preparacion del model para el test a entregar

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


mejores_hiperparametros = {
    'min_samples_split': 5,
    'min_samples_leaf': 4,
    'max_features': None,
    'max_depth': 40,
    'class_weight': 'balanced_subsample',
    'bootstrap': True,
    'n_estimators': 200,
     'random_state': 123
}


model = RandomForestClassifier(**mejores_hiperparametros)


y_train = y_train_raw
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

f1 = f1_score(y_pred=y_pred, y_true=y_test_raw)

print("F1-score del RandomForestClassifier en el conjunto de prueba:", f1)

F1-score del RandomForestClassifier en el conjunto de prueba: 0.6044444444444445


## Prediccion

In [ ]:
test = pd.read_parquet("/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/dataset_v2/test.parq", engine="pyarrow")


In [ ]:
test.head(10)

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum
0,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:spam,1,7696
1,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:exploit,1,7696
2,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:bruteforce,1,7696
3,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:bruteforce,1,7696
4,2023-07-31 07:17:51+00:00,US,396982.0,GOOGLE-CLOUD-PLATFORM,US,53667.0,PONYNET,http:bruteforce,3,7543
5,2023-07-31 07:17:51+00:00,US,396982.0,GOOGLE-CLOUD-PLATFORM,US,53667.0,PONYNET,http:spam,3,7543
6,2023-07-31 07:17:51+00:00,US,396982.0,GOOGLE-CLOUD-PLATFORM,US,53667.0,PONYNET,http:exploit,3,7543
7,2023-07-31 07:17:57+00:00,DK,51468.0,One.com A/S,IN,14061.0,DIGITALOCEAN-ASN,http:exploit,7,7696
8,2023-07-31 07:17:57+00:00,DK,51468.0,One.com A/S,IN,14061.0,DIGITALOCEAN-ASN,http:spam,7,7696
9,2023-07-31 07:17:57+00:00,DK,51468.0,One.com A/S,IN,14061.0,DIGITALOCEAN-ASN,http:bruteforce,7,7696


# Preaparacion del dataset a predecir

In [ ]:
attack_types_df = (
    test.attack_type.str.split(":", expand=True)
    .rename(columns={0: "service", 1: "type"})
    .set_index(test["attacker_ip_enum"])
)

one_hot_attack_service_test = pd.get_dummies(

    attack_types_df.reset_index()
    .drop_duplicates(subset=["attacker_ip_enum", "service"])
    .set_index("attacker_ip_enum")["service"]

)
one_hot_attack_service_test = one_hot_attack_service_test.groupby("attacker_ip_enum").sum()

one_hot_attack_service_test = (one_hot_attack_service_test >= 1).astype(int)

one_hot_attack_service_test.rename(columns={'unknown': 'unknown_service'}, inplace=True)

one_hot_attack_service_test.head()

,database,ftp,http,pop3/imap,sip,smb,ssh,tcp,telnet,unknown_service,windows
attacker_ip_enum,,,,,,,,,,,
5,0,0,1,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,1,0,0,0
21,0,0,1,0,0,0,0,0,0,0,0
29,0,0,1,0,0,0,0,0,0,0,0
33,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
one_hot_attack_types_test = pd.get_dummies(
    attack_types_df["type"],
    # sparse=True
)
one_hot_attack_types_test = one_hot_attack_types_test.groupby("attacker_ip_enum").sum()

one_hot_attack_types_test = one_hot_attack_types_test / one_hot_attack_types_test.sum(
    1
).values.reshape(-1, 1)

one_hot_attack_types_test.rename(columns={'unknown': 'unknown_type'}, inplace=True)

one_hot_attack_types_test.head(10)


,bruteforce,crawl,exploit,scan,spam,unknown_type
attacker_ip_enum,,,,,,
5,0.284848,0.0,0.321212,0.060606,0.333333,0.0
7,0.990000,0.0,0.000000,0.010000,0.000000,0.0
21,0.000000,0.0,0.000000,1.000000,0.000000,0.0
29,0.187500,0.0,0.437500,0.062500,0.312500,0.0
33,0.000000,0.0,0.000000,0.500000,0.500000,0.0
37,0.153846,0.0,0.564103,0.000000,0.282051,0.0
43,0.023810,0.0,0.261905,0.142857,0.571429,0.0
47,0.000000,0.0,0.500000,0.000000,0.500000,0.0
48,0.000000,0.0,0.043478,0.478261,0.478261,0.0


In [ ]:
test['day_of_week'] = pd.to_datetime(test['attack_time']).dt.day_name()
days_test = test[['attacker_ip_enum', 'day_of_week']].set_index('attacker_ip_enum')

one_hot_days_test = pd.get_dummies(

    days_test.reset_index()
    .set_index("attacker_ip_enum")["day_of_week"]
    # ,sparse=True
)

one_hot_days_test = one_hot_days_test.groupby("attacker_ip_enum").sum()

one_hot_days_test = one_hot_days_test / one_hot_days_test.sum(
    1
).values.reshape(-1, 1)
one_hot_days_test.head(10)


,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
attacker_ip_enum,,,,,,,
5,0.115152,0.200000,0.090909,0.127273,0.072727,0.193939,0.200000
7,0.005000,0.010000,0.000000,0.960000,0.015000,0.000000,0.010000
21,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
29,0.000000,0.125000,0.437500,0.000000,0.000000,0.250000,0.187500
33,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
37,0.307692,0.000000,0.000000,0.000000,0.692308,0.000000,0.000000
43,0.238095,0.000000,0.690476,0.071429,0.000000,0.000000,0.000000
47,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
48,0.000000,0.608696,0.000000,0.391304,0.000000,0.000000,0.000000


In [ ]:
test['hour_of_day'] = pd.to_datetime(test['attack_time']).dt.hour
hour_test_df = test[['attacker_ip_enum', 'hour_of_day']].set_index('attacker_ip_enum')

bins = range(0, 25, 2)  # Rangos de 4 horas: 0-3, 3-6, ..., 21-24

# Crea una nueva columna 'hour_range' que asigna cada hora a su rango correspondiente
hour_test_df['hour_range'] = pd.cut(hour_test_df['hour_of_day'], bins=bins, labels=False, right=False)

hour_test_df = hour_test_df.drop(columns=['hour_of_day'])

one_hot_hours_test = pd.get_dummies(

    hour_test_df.reset_index()
    .set_index("attacker_ip_enum")["hour_range"]
    # ,sparse=True
)
one_hot_hours_test = one_hot_hours_test.groupby("attacker_ip_enum").sum()




In [ ]:
one_hot_hours_test.head()

,0,1,2,3,4,5,6,7,8,9,10,11
attacker_ip_enum,,,,,,,,,,,,
5,18,20,18,18,8,8,6,6,12,12,18,21
7,1,3,0,169,24,2,0,0,0,1,0,0
21,0,0,0,0,0,0,0,0,0,0,0,21
29,10,0,0,4,0,0,0,2,0,0,0,0
33,0,0,0,0,0,0,6,0,0,0,0,0


In [ ]:
one_hot_hours_test.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype='int64')

In [ ]:
# We create a df label df which contains one row per attacker IP
test_df = test.drop_duplicates(["attacker_ip_enum"]).set_index("attacker_ip_enum").loc[:, ['watcher_as_num','attacker_as_num','attacker_country','watcher_country']]
#Conviero los '...as_num' a categorica para luego poder aplicarle un encoding acorde
test_df['watcher_as_num'] = test_df['watcher_as_num'].astype('category')
test_df['attacker_as_num'] = test_df['attacker_as_num'].astype('category')

In [ ]:
X_test_df = pd.concat(
    [
        one_hot_attack_service_test,
        one_hot_attack_types_test,
        one_hot_days_test,
        one_hot_hours_test,
        ports_features_df.set_index("attacker_ip_enum"),#mas esto hacer q tenga menos valores
        protocols_features_df.set_index("attacker_ip_enum"),#masmas esto hacer q tenga menos valores
        test_df,
    ],
    axis=1,
    join="inner",
)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

X_test_df['attacker_as_num'] = X_test_df['attacker_as_num'].map(means_attacker_as_num)
X_test_df['watcher_as_num'] = X_test_df['watcher_as_num'].map(means_watcher_as_num)

In [ ]:
# Imputar los valores nulos
X_test_df[columns_with_nans] = imputer.transform(X_test_df[columns_with_nans])

In [ ]:
X_test_df.count()

database            49420
ftp                 49420
http                49420
pop3/imap           49420
sip                 49420
smb                 49420
ssh                 49420
tcp                 49420
telnet              49420
unknown_service     49420
windows             49420
bruteforce          49420
crawl               49420
exploit             49420
scan                49420
spam                49420
unknown_type        49420
Friday              49420
Monday              49420
Saturday            49420
Sunday              49420
Thursday            49420
Tuesday             49420
Wednesday           49420
0                   49420
1                   49420
2                   49420
3                   49420
4                   49420
5                   49420
6                   49420
7                   49420
8                   49420
9                   49420
10                  49420
11                  49420
port_1080           49420
port_110            49420
port_1194   

In [ ]:
from category_encoders.binary import BinaryEncoder
from sklearn.impute import SimpleImputer

# Crear una nueva columna combinando "attacker_country" y "watcher_country"
X_test_df['combined_countries'] = X_test_df['attacker_country'] + '-' + X_test_df['watcher_country']

# Aplicar el BinaryEncoder a la nueva columna
X_test_df = encoder.transform(X_test_df)

In [ ]:
X_test_df.columns = X_test_df.columns.astype(str)
X_test_df = X_test_df[X_train_enc.columns]
#X_test = sc.transform(X_test_df)

In [ ]:
X_test_df.columns

Index(['database', 'ftp', 'http', 'pop3/imap', 'sip', 'smb', 'ssh', 'tcp',
       'telnet', 'unknown_service', 'windows', 'bruteforce', 'crawl',
       'exploit', 'scan', 'spam', 'unknown_type', 'Friday', 'Monday',
       'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'port_1080',
       'port_110', 'port_1194', 'port_143', 'port_1701', 'port_1723',
       'port_20', 'port_21', 'port_22', 'port_23', 'port_25', 'port_500',
       'port_53', 'port_8080', 'open_ports_count', 'protocol_tcp',
       'protocol_udp', 'protocols_count', '0', '1', '2', '3', '4', '5', '6',
       '7', '8', '9', '10', '11', 'watcher_as_num', 'attacker_as_num',
       'attacker_country_0', 'attacker_country_1', 'attacker_country_2',
       'attacker_country_3', 'attacker_country_4', 'attacker_country_5',
       'attacker_country_6', 'attacker_country_7', 'watcher_country_0',
       'watcher_country_1', 'watcher_country_2', 'watcher_country_3',
       'watcher_country_4', 'watcher_country_5', 'watcher_co

## Entrega de predicciones

In [ ]:
prediction_model1 = pd.Series(model.predict(X_test_df), index=X_test_df.index).rename("prediction_model1")

In [ ]:
prediction_model1.head()

attacker_ip_enum
5     0
7     0
21    0
29    0
33    0
Name: prediction_model1, dtype: int8

In [ ]:
prediction_model1.to_csv('/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/Predictions/prediction_model1.csv')

In [ ]:
len(prediction_model1)

49420

https://drive.google.com/drive/folders/1VHyzVuEJ1dy1rgDXhN9Vp3kqt3lGltiJ?usp=sharing